<h1 style='font-size:40px'> Apache Spark's Structured APIs</h1>

<h2 style='font-size:30px'> Key Merits and Beneftis</h2>

In [7]:
# Criando o nosso primeiro DataFrame dentro do Spark.
# Vamos aproveitar a situação e fazer algumas operações.
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
spark = SparkSession.builder.appName('Chapter3').getOrCreate()

# O método createDataFrame demanda uma lista de tuplas para que cada linha seja construída.
# Como segundo argumento, uma lista com o nome das colunas pode ser passada.
data_df = spark.createDataFrame([('Brooke', 20), ('Denny', 31), ('Jules', 30),
                                ('TD', 35), ('Brooke', 25)],['name', 'age'])

data_df.show()

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 31|
| Jules| 30|
|    TD| 35|
|Brooke| 25|
+------+---+



In [9]:
# Agrupando os dados conforme o nome e extraindo a idade média.

# A sintaxe é um pouco diferente da do pandas, o método agg com um outro método de agregação como argumento deve acompahar 'groupBy'.
data_df.groupBy('name').agg(avg('age')).show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Denny|    31.0|
| Jules|    30.0|
|    TD|    35.0|
+------+--------+



<h2 style='font-size:30px'> The DataFrame API</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Os DataFrames do PySpark são como os do pandas. Têm estrutura de tabelas e permitem a nós fazermos operações sobre os seus dados.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Spark's Basic Data Types</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O Spark possui uma série de tipos de dados que podem ser designados às colunas dos DataFrames.
        </li>
    </ul>
</div>

<center> 
    <img src='data_types1.png'>
</center>

<h3 style='font-size:30px;font-style:italic'> Schemas and Creating DataFrames</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            O <em> schema</em> de um DataFrame é a relação do nome de suas colunas com os seus respectivos tipos de dados.
        </li>
        <li> 
            O Spark é capaz de inferir o schema de uma tabela, mas isso pode ser demorado e demandar uma alta capacidade de processamento do computador. Por isso, é recomendável que nós façamos essa tarefa.
        </li>
    </ul>
</div>

In [15]:
# Vamos definir o schema do DataFrame que iremos construir.

# Do jeito mais formal. O jeito de se construir lembra o das Pipelines do scikit-learn.
from pyspark.sql.types import *
schema = StructType([
         StructField('author', StringType(), False),
         StructField('title', StringType(), False),
        StructField('pages', IntegerType(), False)])

In [24]:
# Do jeito mais simples.
schema = '''`Id` INT, `First` STRING, `Last` STRING, `URL` STRING, 
        `Published` STRING, `Hits` INT, `Campaigns` ARRAY<STRING>'''
data=[[1, 'Jules', 'Damji', 'https://www.oi1.com', '1/4/2016', 4535, ['twitter', 'LinkedIn']],
    [2, 'Brooke', 'Wenig', 'htps://oi2.com', '5/5/2018', 8908, ['twitter', 'LinkedIn']],
     [3, 'Tathagata', 'Das', 'htps://oi3.com', '5/12/2018', 10568,['twitter', 'FB']]]

# Criando o DataFrame.
blogs_df = spark.createDataFrame(data, schema)

# Pelo printSchema, é possível enxergar que as colunas têm o nome e o tipo de dados desejados.
blogs_df.printSchema(),blogs_df.show()

root
 |-- Id: integer (nullable = true)
 |-- First: string (nullable = true)
 |-- Last: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- Published: string (nullable = true)
 |-- Hits: integer (nullable = true)
 |-- Campaigns: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---+---------+-----+-------------------+---------+-----+-------------------+
| Id|    First| Last|                URL|Published| Hits|          Campaigns|
+---+---------+-----+-------------------+---------+-----+-------------------+
|  1|    Jules|Damji|https://www.oi1.com| 1/4/2016| 4535|[twitter, LinkedIn]|
|  2|   Brooke|Wenig|     htps://oi2.com| 5/5/2018| 8908|[twitter, LinkedIn]|
|  3|Tathagata|  Das|     htps://oi3.com|5/12/2018|10568|      [twitter, FB]|
+---+---------+-----+-------------------+---------+-----+-------------------+



(None, None)

<h3 style='font-size:30px;font-style:italic'> Columns and Expressions </h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Assim como no pandas e no SQL, os DataFrames Spark admitem que o usuário faça expressões com as suas colunas.
        </li>
    </ul>
</div>

In [54]:
from pyspark.sql.functions import *

# Os blogueiros têm mais de 1000 hits?
blogs_df.select('First',expr('Hits >1000')).show()

# Divida 'Hits' por 2;
blogs_df.select(expr('Hits /2')).show()

# Podemos fazer a mesma operação com o objeto 'col'.
blogs_df.select(col('Hits')/2).show()

# Uma nova coluna com nome e sobrenome do blogueiro.
blogs_df.withColumn('Full Name', concat(expr('First'), expr('Last'))).show()

# Reordenando o DF na ordem inversa dos ID's.
blogs_df.sort(col('Id').desc()).show()

+---------+-------------+
|    First|(Hits > 1000)|
+---------+-------------+
|    Jules|         true|
|   Brooke|         true|
|Tathagata|         true|
+---------+-------------+

+----------+
|(Hits / 2)|
+----------+
|    2267.5|
|    4454.0|
|    5284.0|
+----------+

+----------+
|(Hits / 2)|
+----------+
|    2267.5|
|    4454.0|
|    5284.0|
+----------+

+---+---------+-----+-------------------+---------+-----+-------------------+------------+
| Id|    First| Last|                URL|Published| Hits|          Campaigns|   Full Name|
+---+---------+-----+-------------------+---------+-----+-------------------+------------+
|  1|    Jules|Damji|https://www.oi1.com| 1/4/2016| 4535|[twitter, LinkedIn]|  JulesDamji|
|  2|   Brooke|Wenig|     htps://oi2.com| 5/5/2018| 8908|[twitter, LinkedIn]| BrookeWenig|
|  3|Tathagata|  Das|     htps://oi3.com|5/12/2018|10568|      [twitter, FB]|TathagataDas|
+---+---------+-----+-------------------+---------+-----+-------------------+----------

<h3 style='font-size:30px;font-style:italic'> Rows </h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
           Row é um objeto do pyspark que representa uma linha de DataFrame. Pode conter dados de diversas espécies.
        </li>
    </ul>
</div>

In [57]:
from pyspark.sql import Row
blog_row = Row(6, 'Reynold', 'Xin', 'htps://oi4.com', 255568, '3/2/2015', ['twitter', 'LinkedIn'])

# Podemos acessar o valor de uma das colunas da linha por meio de seu íindice.
blog_row[1]

'Reynold'

In [61]:
# Uma coleção de Rows pode dar origem a um DataFrame.
rows = [Row('Felipe', 'Veiga', 1),Row('Luiz', 'Villar', 2), Row('Eduardo', 'Rodrigues', 3)]
spark.createDataFrame(rows, '`First`STRING, `Last`STRING, `ID`INT').show()

+-------+---------+---+
|  First|     Last| ID|
+-------+---------+---+
| Felipe|    Veiga|  1|
|   Luiz|   Villar|  2|
|Eduardo|Rodrigues|  3|
+-------+---------+---+



<h3 style='font-size:30px;font-style:italic'> Common DataFrame Operations</h3>
<div> 
    <ul style='font-size:20px'> 
        <li> 
        </li>
    </ul>
</div>

<p style='color:red'> Common DataFrame Operations

<a href='https://databricks.com/notebooks/gallery/SanFranciscoFireCallsAnalysis.html'> Link do dataset de chamados de incêndio</a>